In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Dubai Satellite Imagery Semantic Segmentation
Humans in the Loop has published an open access dataset annotated for a joint project with the Mohammed Bin Rashid Space Center in Dubai, the UAE.

The dataset consists of aerial imagery of Dubai obtained by MBRSC satellites and annotated with pixel-wise semantic segmentation in 6 classes. The images were segmented by the trainees of the Roia Foundation in Syria.

Original Dataset Link: https://humansintheloop.org/resources/datasets/semantic-segmentation-dataset/

# Installing & Importing Libraries

In [2]:
import pickle
import numpy as np
import pandas as pd
from PIL import Image
import albumentations as A
from IPython.display import SVG
import matplotlib.pyplot as plt
%matplotlib inline
import os, re, sys, random, shutil, cv2

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, Nadam
from tensorflow.keras import applications, optimizers
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.applications.resnet50 import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.utils import model_to_dot, plot_model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger, LearningRateScheduler
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, ZeroPadding2D, Dropout

In [3]:
def visualize(image, mask, original_image=None, original_mask=None):
    fontsize = 16

    if original_image is None and original_mask is None:
        f, ax = plt.subplots(2, 1, figsize=(10, 10), squeeze=True)
        f.set_tight_layout(h_pad=5, w_pad=5)

        ax[0].imshow(image)
        ax[1].imshow(mask)
    else:
        f, ax = plt.subplots(2, 2, figsize=(16, 12), squeeze=True)
        plt.tight_layout(pad=0.2, w_pad=1.0, h_pad=0.01)

        ax[0, 0].imshow(original_image)
        ax[0, 0].set_title('Original Image', fontsize=fontsize)

        ax[1, 0].imshow(original_mask)
        ax[1, 0].set_title('Original Mask', fontsize=fontsize)

        ax[0, 1].imshow(image)
        ax[0, 1].set_title('Transformed Image', fontsize=fontsize)

        ax[1, 1].imshow(mask)
        ax[1, 1].set_title('Transformed Mask', fontsize=fontsize)

    plt.savefig('sample_augmented_image.png', facecolor= 'w', transparent= False, bbox_inches= 'tight', dpi= 100)

# Working with Augmented Dataset

In [4]:
train_images = "/content/drive/MyDrive/Colab Notebooks/datasets/Aug/Images/Train/"
train_masks = "/content/drive/MyDrive/Colab Notebooks/datasets/Aug/Masks/Train/"
val_images = "/content/drive/MyDrive/Colab Notebooks/datasets/Aug/Images/Validation/"
val_masks = "/content/drive/MyDrive/Colab Notebooks/datasets/Aug/Masks/Validation/"

In [5]:
file_names = np.sort(os.listdir(train_images ))
file_names = np.char.split(file_names, '.')
filenames = np.array([])
for i in range(len(file_names)):
    filenames = np.append(filenames, file_names[i][0])

In [6]:
def show_data(files, original_images_dir, label_images_dir):

    for file in files:
        fig, axs = plt.subplots(1, 2, figsize=(15, 6), constrained_layout=True)

        axs[0].imshow(cv2.resize(cv2.imread(original_images_dir+str(file)+'.jpg'), (2000,1400)))
        axs[0].set_title('Original Image', fontdict = {'fontsize':14, 'fontweight': 'medium'})
        axs[0].set_xticks(np.arange(0, 2001, 200))
        axs[0].set_yticks(np.arange(0, 1401, 200))
        axs[0].grid(False)
        axs[0].axis(True)

        semantic_label_image = cv2.imread(label_images_dir+str(file)+'.png')
       # semantic_label_image = cv2.cvtColor(semantic_label_image, cv2.COLOR_BGR2RGB)
        semantic_label_image = cv2.resize(semantic_label_image, (2000,1400))
        axs[1].imshow(semantic_label_image)
        axs[1].set_title('Semantic Segmentation Mask', fontdict = {'fontsize':14, 'fontweight': 'medium'})
        axs[1].set_xticks(np.arange(0, 2001, 200))
        axs[1].set_yticks(np.arange(0, 1401, 200))
        axs[1].grid(False)
        axs[1].axis(True)

        plt.savefig('./sample_'+file, facecolor= 'w', transparent= False, bbox_inches= 'tight', dpi= 100)
        plt.show()



#files = ['patch_5454', 'patch_5564', 'patch_5741', 'patch_5965', 'patch_6066', 'patch_6955', 'patch_7259',]
#show_data(files, train_images, train_masks)

In [7]:
import pandas as pd

# Define the class_dict data as a list of dictionaries
class_dict_data = [
    {'name': 'building', 'r': 60, 'g': 16, 'b': 152},
    {'name': 'land', 'r': 132, 'g': 41, 'b': 246},
    {'name': 'road', 'r': 110, 'g': 193, 'b': 228},
    {'name': 'vegetation', 'r': 254, 'g': 221, 'b': 58},
    {'name': 'water', 'r': 226, 'g': 169, 'b': 41},
    {'name': 'unlabeled', 'r': 155, 'g': 155, 'b': 155}
]

# Create the DataFrame
class_dict_df = pd.DataFrame(class_dict_data)
class_dict_df
# Now, class_dict_df contains the desired DataFrame

,name,r,g,b
0,building,60,16,152
1,land,132,41,246
2,road,110,193,228
3,vegetation,254,221,58
4,water,226,169,41
5,unlabeled,155,155,155


In [8]:
label_names= list(class_dict_df.name)
label_codes = []
r= np.asarray(class_dict_df.r)
g= np.asarray(class_dict_df.g)
b= np.asarray(class_dict_df.b)

for i in range(len(class_dict_df)):
    label_codes.append(tuple([r[i], g[i], b[i]]))

label_codes, label_names

([(60, 16, 152),
  (132, 41, 246),
  (110, 193, 228),
  (254, 221, 58),
  (226, 169, 41),
  (155, 155, 155)],
 ['building', 'land', 'road', 'vegetation', 'water', 'unlabeled'])

# Create Useful Label & Code Conversion Dictionaries

These will be used for:

* One hot encoding the mask labels for model training
* Decoding the predicted labels for interpretation and visualization

In [9]:
code2id = {v:k for k,v in enumerate(label_codes)}
id2code = {k:v for k,v in enumerate(label_codes)}

name2id = {v:k for k,v in enumerate(label_names)}
id2name = {k:v for k,v in enumerate(label_names)}

In [10]:
id2code

{0: (60, 16, 152),
 1: (132, 41, 246),
 2: (110, 193, 228),
 3: (254, 221, 58),
 4: (226, 169, 41),
 5: (155, 155, 155)}

In [11]:
id2name

{0: 'building',
 1: 'land',
 2: 'road',
 3: 'vegetation',
 4: 'water',
 5: 'unlabeled'}

# Define Functions for One Hot Encoding RGB Labels & Decoding Encoded Predictions

In [12]:
def rgb_to_onehot(rgb_image, colormap = id2code):
    '''Function to one hot encode RGB mask labels
        Inputs:
            rgb_image - image matrix (eg. 256 x 256 x 3 dimension numpy ndarray)
            colormap - dictionary of color to label id
        Output: One hot encoded image of dimensions (height x width x num_classes) where num_classes = len(colormap)
    '''
    num_classes = len(colormap)
    shape = rgb_image.shape[:2]+(num_classes,)
    encoded_image = np.zeros( shape, dtype=np.int8 )
    for i, cls in enumerate(colormap):
        encoded_image[:,:,i] = np.all(rgb_image.reshape( (-1,3) ) == colormap[i], axis=1).reshape(shape[:2])
    return encoded_image


def onehot_to_rgb(onehot, colormap = id2code):
    '''Function to decode encoded mask labels
        Inputs:
            onehot - one hot encoded image matrix (height x width x num_classes)
            colormap - dictionary of color to label id
        Output: Decoded RGB image (height x width x 3)
    '''
    single_layer = np.argmax(onehot, axis=-1)
    output = np.zeros( onehot.shape[:2]+(3,) )
    for k in colormap.keys():
        output[single_layer==k] = colormap[k]
    return np.uint8(output)

# Creating Custom Image Data Generators
## Defining Data Generators

# Custom Image Data Generators for Creating Batches of Frames and Masks

# Model

In [13]:
train_images = "/content/drive/MyDrive/Colab Notebooks/datasets/Aug/Images/Train/"
train_masks = "/content/drive/MyDrive/Colab Notebooks/datasets/Aug/Masks/Train/"
val_images = "/content/drive/MyDrive/Colab Notebooks/datasets/Aug/Images/Validation/"
val_masks = "/content/drive/MyDrive/Colab Notebooks/datasets/Aug/Masks/Validation/"


In [14]:
from keras.utils import to_categorical

# List all image and mask files for training and validation
train_image_files = os.listdir(train_images)
train_mask_files = os.listdir(train_masks)
val_image_files = os.listdir(val_images)
val_mask_files = os.listdir(val_masks)

# Define a generator to load data batch by batch
def data_generator(image_files, mask_files, batch_size, num_classes, colormap):
    num_samples = len(image_files)
    while True:
        batch_indices = np.random.choice(num_samples, batch_size, replace=False)
        batch_images = []
        batch_masks = []
        for index in batch_indices:
            image = cv2.imread(os.path.join(train_images, image_files[index]))
            mask = cv2.imread(os.path.join(train_masks, mask_files[index]), cv2.IMREAD_COLOR)

            # Preprocess and augment data as needed

            # One-hot encode the mask using the provided functions
            mask_onehot = rgb_to_onehot(mask, colormap)

            # Add data to batch_images and batch_masks
            batch_images.append(image)
            batch_masks.append(mask_onehot)

        yield np.array(batch_images), np.array(batch_masks)
# Define a generator to load data batch by batch
def data_generator2(image_files, mask_files, batch_size, num_classes, colormap):
    num_samples = len(image_files)
    while True:
        batch_indices = np.random.choice(num_samples, batch_size, replace=False)
        batch_images = []
        batch_masks = []
        for index in batch_indices:
            image = cv2.imread(os.path.join(val_images, image_files[index]))
            mask = cv2.imread(os.path.join(val_masks, mask_files[index]), cv2.IMREAD_COLOR)

            # Preprocess and augment data as needed

            # One-hot encode the mask using the provided functions
            mask_onehot = rgb_to_onehot(mask, colormap)

            # Add data to batch_images and batch_masks
            batch_images.append(image)
            batch_masks.append(mask_onehot)

        yield np.array(batch_images), np.array(batch_masks)


input_shape = (256, 256, 3)  # Adjust the input shape based on your images
num_classes = len(id2code)  # Make sure to define your colormap




In [15]:
batch_size = 16
num_train_samples = len(np.sort(os.listdir(train_images)))
num_val_samples = len(np.sort(os.listdir(val_images)))
steps_per_epoch = np.ceil(float(num_train_samples) / float(2*batch_size))
print('steps_per_epoch: ', steps_per_epoch)
validation_steps = np.ceil(float(num_val_samples) / float(2*batch_size))
print('validation_steps: ', validation_steps)

steps_per_epoch:  160.0
validation_steps:  35.0


## InceptionResNetV2 UNet

In [16]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose
from keras.layers import concatenate, BatchNormalization, Dropout, Lambda

In [17]:
def multi_unet_model(n_classes=6, image_height=256, image_width=256, image_channels=3):

  inputs = Input((image_height, image_width, image_channels))

  source_input = inputs

  c1 = Conv2D(32, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(source_input)
  c1 = Dropout(0.2)(c1)
  c1 = Conv2D(32, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(c1)
  p1 = MaxPooling2D((2,2))(c1)

  c2 = Conv2D(64, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(p1)
  c2 = Dropout(0.2)(c2)
  c2 = Conv2D(64, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(c2)
  p2 = MaxPooling2D((2,2))(c2)

  c3 = Conv2D(128, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(p2)
  c3 = Dropout(0.2)(c3)
  c3 = Conv2D(128, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(c3)
  p3 = MaxPooling2D((2,2))(c3)

  c4 = Conv2D(256, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(p3)
  c4 = Dropout(0.2)(c4)
  c4 = Conv2D(256, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(c4)
  p4 = MaxPooling2D((2,2))(c4)

  c5 = Conv2D(512, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(p4)
  c5 = Dropout(0.2)(c5)
  c5 = Conv2D(512, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(c5)

  u6 = Conv2DTranspose(256, (2,2), strides=(2,2), padding="same")(c5)
  u6 = concatenate([u6, c4])
  c6 = Conv2D(256, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(u6)
  c6 = Dropout(0.2)(c6)
  c6 = Conv2D(256, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(c6)

  u7 = Conv2DTranspose(128, (2,2), strides=(2,2), padding="same")(c6)
  u7 = concatenate([u7, c3])
  c7 = Conv2D(128, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(u7)
  c7 = Dropout(0.2)(c7)
  c7 = Conv2D(128, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(c7)

  u8 = Conv2DTranspose(64, (2,2), strides=(2,2), padding="same")(c7)
  u8 = concatenate([u8, c2])
  c8 = Conv2D(64, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(u8)
  c8 = Dropout(0.2)(c8)
  c8 = Conv2D(64, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(c8)

  u9 = Conv2DTranspose(32, (2,2), strides=(2,2), padding="same")(c8)
  u9 = concatenate([u9, c1], axis=3)
  c9 = Conv2D(32, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(u9)
  c9 = Dropout(0.2)(c9)
  c9 = Conv2D(32, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(c9)

  outputs = Conv2D(n_classes, (1,1), activation="softmax")(c9)

  model = Model(inputs=[inputs], outputs=[outputs])
  return model

In [18]:
def get_deep_learning_model():
  return multi_unet_model(n_classes=6,
                          image_height=256,
                          image_width=256,
                          image_channels=3)

In [19]:
model = get_deep_learning_model()

In [20]:
K.clear_session()

def dice_coef(y_true, y_pred):
    return (2. * K.sum(y_true * y_pred) + 1.) / (K.sum(y_true) + K.sum(y_pred) + 1.)

#model.summary()

In [21]:
model.compile(optimizer=Adam(learning_rate = 0.0001), loss='categorical_crossentropy', metrics=[dice_coef, "accuracy"])


In [22]:
#SVG(model_to_dot(model).create(prog='dot', format='svg'))
#plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True, expand_nested=True)

In [23]:
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 **(epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(0.0001, 60)

lr_scheduler = LearningRateScheduler(
    exponential_decay_fn,
    verbose=1
)

checkpoint = ModelCheckpoint(
    filepath = 'InceptionResNetV2-UNet.h5',
    save_best_only = True,
#     save_weights_only = False,
    monitor = 'val_loss',
    mode = 'auto',
    verbose = 1
)

earlystop = EarlyStopping(
    monitor = 'val_loss',
    min_delta = 0.001,
    patience = 12,
    mode = 'auto',
    verbose = 1,
    restore_best_weights = True
)

csvlogger = CSVLogger(
    filename= "model_training.csv",
    separator = ",",
    append = False
)

callbacks = [checkpoint, earlystop, csvlogger, lr_scheduler]

In [24]:
#from tensorflow.keras.callbacks import Callback
#import os

# Create a directory to save model weights
#weights_save_dir = 'model_weights/'
#if not os.path.exists(weights_save_dir):
#    os.makedirs(weights_save_dir)

# Define a custom callback to save model weights every 5 epochs
#class SaveModelWeightsOnEpochsCallback(Callback):
#    def __init__(self, filepath, save_every_n_epochs):
#        super().__init__()
#        self.filepath = filepath
#        self.save_every_n_epochs = save_every_n_epochs

#    def on_epoch_end(self, epoch, logs=None):
#        if (epoch + 1) % self.save_every_n_epochs == 0:
#            self.model.save_weights(self.filepath.format(epoch=epoch+1))

# Define the checkpoint callback to save model weights every 5 epochs
#checkpoint_callback = SaveModelWeightsOnEpochsCallback(
#    filepath=os.path.join(weights_save_dir, 'model_weights_epoch_{epoch:02d}.h5'),
#    save_every_n_epochs=5  # Save weights every 5 epochs
#)


In [25]:
#model.load_weights('model_weights_epoch_05.h5')  # Replace with the path to your saved weights
#model.compile(optimizer=Adam(learning_rate = 0.0001), loss='categorical_crossentropy', metrics=[dice_coef, "accuracy"])

In [ ]:
history = model.fit(
    data_generator(train_image_files, train_mask_files, batch_size, num_classes, id2code),
    steps_per_epoch=steps_per_epoch, epochs=10,
    validation_data=data_generator2(val_image_files, val_mask_files, batch_size, num_classes, id2code),
    validation_steps=validation_steps,
    #callbacks=callbacks,
    #callbacks=[checkpoint_callback],
    #use_multiprocessing=True,
    verbose=1
)

Epoch 1/10
160/160 [==============================] - 1105s 7s/step - loss: 44.2743 - dice_coef: 0.2787 - accuracy: 0.2814 - val_loss: 1.8539 - val_dice_coef: 0.1693 - val_accuracy: 0.1365
Epoch 2/10
160/160 [==============================] - 646s 4s/step - loss: 1.9424 - dice_coef: 0.1734 - accuracy: 0.4062 - val_loss: 1.7825 - val_dice_coef: 0.1703 - val_accuracy: 0.4823
Epoch 3/10
160/160 [==============================] - 395s 2s/step - loss: 1.8068 - dice_coef: 0.1754 - accuracy: 0.4888 - val_loss: 1.7413 - val_dice_coef: 0.1746 - val_accuracy: 0.4766
Epoch 4/10
160/160 [==============================] - 250s 2s/step - loss: 1.7488 - dice_coef: 0.1820 - accuracy: 0.5043 - val_loss: 1.7079 - val_dice_coef: 0.1825 - val_accuracy: 0.5089
Epoch 5/10
 71/160 [============>.................] - ETA: 1:02 - loss: 1.7252 - dice_coef: 0.1869 - accuracy: 0.5044

In [ ]:
df_result = pd.DataFrame(history.history)
df_result

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(40, 5))
ax = ax.ravel()
metrics = ['Dice Coefficient', 'Accuracy', 'Loss', 'Learning Rate']

for i, met in enumerate(['dice_coef', 'accuracy', 'loss', 'lr']):
    if met != 'lr':
        ax[i].plot(history.history[met])
        ax[i].plot(history.history['val_' + met])
        ax[i].set_title('{} vs Epochs'.format(metrics[i]), fontsize=16)
        ax[i].set_xlabel('Epochs')
        ax[i].set_ylabel(metrics[i])
        ax[i].set_xticks(np.arange(0,45,4))
        ax[i].legend(['Train', 'Validation'])
        ax[i].xaxis.grid(True, color = "lightgray", linewidth = "0.8", linestyle = "-")
        ax[i].yaxis.grid(True, color = "lightgray", linewidth = "0.8", linestyle = "-")
    else:
        ax[i].plot(history.history[met])
        ax[i].set_title('{} vs Epochs'.format(metrics[i]), fontsize=16)
        ax[i].set_xlabel('Epochs')
        ax[i].set_ylabel(metrics[i])
        ax[i].set_xticks(np.arange(0,45,4))
        ax[i].xaxis.grid(True, color = "lightgray", linewidth = "0.8", linestyle = "-")
        ax[i].yaxis.grid(True, color = "lightgray", linewidth = "0.8", linestyle = "-")

plt.savefig('model_metrics_plot.png', facecolor= 'w',transparent= False, bbox_inches= 'tight', dpi= 150)

In [ ]:
model.load_weights("./InceptionResNetV2-UNet.h5")

In [ ]:
testing_gen = ValAugmentGenerator(val_images_dir = val_images, val_masks_dir = val_masks, target_size = (512, 512))

In [ ]:
!mkdir predictions

In [ ]:
count = 0
for i in range(2):
    batch_img,batch_mask = next(testing_gen)
    pred_all= model.predict(batch_img)
    np.shape(pred_all)

    for j in range(0,np.shape(pred_all)[0]):
        count += 1
        fig = plt.figure(figsize=(20,8))

        ax1 = fig.add_subplot(1,3,1)
        ax1.imshow(batch_img[j])
        ax1.set_title('Input Image', fontdict={'fontsize': 16, 'fontweight': 'medium'})
        ax1.grid(False)

        ax2 = fig.add_subplot(1,3,2)
        ax2.set_title('Ground Truth Mask', fontdict={'fontsize': 16, 'fontweight': 'medium'})
        ax2.imshow(onehot_to_rgb(batch_mask[j],id2code))
        ax2.grid(False)

        ax3 = fig.add_subplot(1,3,3)
        ax3.set_title('Predicted Mask', fontdict={'fontsize': 16, 'fontweight': 'medium'})
        ax3.imshow(onehot_to_rgb(pred_all[j],id2code))
        ax3.grid(False)

        plt.savefig('./predictions/prediction_{}.png'.format(count), facecolor= 'w', transparent= False, bbox_inches= 'tight', dpi= 200)
        plt.show()

In [ ]:
!zip -r predictions.zip "./predictions"

In [ ]:
!rm -rf './predictions/'